# Monte-Carlo-Integration

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Auswertung von ein- und mehrdimensionalen Integralen unter Verwendung von SciPy-Funktionen sowie mittels der Monte-Carlo-Methode gegenübergestellt. In einer Dimension wird das Integral

$$I = \int_{-\infty}^{+\infty}\text{d}x \frac{x^2}{\sqrt{2\pi}}\exp\left(-\frac{x^2}{2} \right)$$

betrachtet. Dieses Integral lässt sich analytisch auswerten und ergibt $I=1$, so dass wir den tatsächlichen Fehler des numerisch erhaltenen Integral bestimmen können. Für mehrdimensionale Integrale betrachten wir

$$I_d = \int_{-\infty}^{+\infty}\frac{\text{d}x_1}{\sqrt{2\pi}}\ldots\int_{-\infty}^{+\infty}\frac{\text{d}x_d}{\sqrt{2\pi}}\left(x_1^2+\ldots+x_d^2\right)\exp\left(-\frac{x_1^2+\ldots+x_d^2}{2} \right)\,.$$

Auch dieses Integral lässt sich analytisch auswerten, und man erhält $I_d=d$. Somit lässt sich auch hier der tatsächliche Fehler des numerisch erhaltenen Integrals bestimmen.

Im ersten Teil des Jupyter-Notebooks werden Funktionen aus dem `integrate`-Modul des SciPy-Pakets verwendet, während im zweiten Teil die Monte-Carlo-Methode verwendet wird. Interessant ist, wie sich im ersten Fall die Anzahl der erforderlichen Stützstellen und damit die Rechenzeit mit zunehmender Dimension verhält. Damit verglichen werden soll die benötigte Rechenzeit und die Genauigkeit der Auswertung mehrdimensionaler Integrale mit Hilfe der Monte-Carlo-Methode.

## Importanweisungen

In [ ]:
import numpy as np
from numpy import linalg as LA
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact

## Integration mit Hilfe von Funktionen aus `scipy.integrate`

### Eindimensionale Integration

Zur Ausführung des Integrals über den in der Funktion `integrand` definierten Integranden wird hier die Funktion `integrate.quad` aus dem SciPy-Paket verwendet. Dabei sind der relative Fehler `relerr` und der absolute Fehler `abserr` anzugeben, wobei nur die schwächere Bedingung zu erfüllen ist. Es kann daher zum Beispiel sinnvoll sein, den absoluten Fehler auf null zu setzen. Durch Setzen des Arguments `full_output` auf `True` erhalten wir detailliertere Informationen über die Integration, wobei wir uns in erster Linie für die Anzahl der verwendeten Stützstellen interessieren, die im Dictionary-Eintrag `info_dict['neval']` enthalten ist.

In [ ]:
def integrand(x):
    return x**2 / np.sqrt(2*np.pi) * np.exp(-x**2/2)

In [ ]:
def integral_quad(relerr, abserr):
    int_result, int_err, info_dict = integrate.quad(
        integrand, -np.inf, np.inf,
        epsabs=abserr, epsrel=relerr, full_output=True)
    return int_result, int_err, info_dict

### Implementierung der Bedienelemente und Ausgabe des Integrationsergebnisses

Mit Hilfe der Schieberegler lassen sich die folgenden beiden Parameter einstellen:
- `abserr`: absoluter Fehler
- `relerr`: relativer Fehler

Neben dem Ergebnis für das Integral und die Fehlerabschätzung werden der tatsächliche relative Fehler, der hier gleich dem absoluten Fehler ist, sowie die Zahl der verwendeten Stützstellen ausgegeben

In [ ]:
widget_dict = {"abserr":
               widgets.FloatLogSlider(
                   value=1e-6, min=-12, max=-3, step=1,
                   description=r"$\epsilon_\text{abs}$"),
               "relerr":
               widgets.FloatLogSlider(
                   value=1e-6, min=-12, max=-3, step=1,
                   description=r"$\epsilon_\text{rel}$")
               }

@interact(**widget_dict)
def print_scipy_1d(abserr, relerr):
    int_result_quad, int_error, info_dict = integral_quad(
        abserr, relerr)
    print(f"{int_result_quad} ± {int_error:8.2e}")
    print("Tatsächlicher relativer Fehler: "
          f"{abs(int_result_quad-1):8.2e}")
    print(f"Zahl der Stützstellen: {info_dict['neval']}")

### Mehrdimensionale Integration

Für die Berechnung des oben angegebenen mehrdimensionalen Integrals wird die Funktion `integrate.nquad` aus dem SciPy-Paket herangezogen. Die Funktion `integrand_ndim` gibt den Integranden an, wobei das Argument `x` der Dimension des Integrals entsprechend $d$ Komponenten besitzt.

In [ ]:
def integrand_ndim(*x):
    r = LA.norm(x)
    return r**2 * np.exp(-r**2/2) / (2*np.pi)**(0.5*len(x))

In [ ]:
def integral_nquad_ndim(abserr, relerr, n_dim):
    ranges = [(-np.inf, np.inf)]*n_dim
    int_result, int_err, info_dict = integrate.nquad(
        integrand_ndim, ranges,
        opts=dict(epsabs=abserr, epsrel=relerr),
        full_output=True)
    return int_result, int_err, info_dict

### Implementierung der Bedienelemente und Ausgabe des Integrationsergebnisses

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_dim`: Dimension des Integrals
- `abserr`: absoluter Fehler
- `relerr`: relativer Fehler

Neben dem Ergebnis für das mehrdimensionale Integral und die Fehlerabschätzung werden der tatsächliche relative Fehler sowie die Zahl der verwendeten Stützstellen ausgegeben. Da die Rechnung je nach Dimension des Integrals etwas länger dauern kann, wird bei Bedarf eine entsprechende Nachricht ausgegeben.

In [ ]:
widget_dict = {"n_dim":
               widgets.IntSlider(
                   value=1, min=1, max=3, step=1,
                   description=r"$n_\text{dim}$"),
               "abserr":
               widgets.FloatLogSlider(
                   value=1e-6, min=-10, max=-3, step=1,
                   description=r"$\epsilon_\text{abs}$"),
               "relerr":
               widgets.FloatLogSlider(
                   value=1e-6, min=-10, max=-3, step=1,
                   description=r"$\epsilon_\text{rel}$")
               }

@interact(**widget_dict)
def print_scipy_nd(n_dim, abserr, relerr):
    out = widgets.Output()
    display(out)
    out.append_stdout("Bitte etwas Geduld...")

    int_result, int_error, info_dict = integral_nquad_ndim(
        abserr, relerr, n_dim)

    out.clear_output()
    with out:
        print(f"{int_result} ± {int_error:8.2e}")
        print("Tatsächlicher relativer Fehler: "
              f"{abs(int_result-n_dim)/n_dim:8.2e}")
        print(f"Zahl der Stützstellen: {info_dict['neval']}")

## Integration mit Hilfe der Monte-Carlo-Methode

### Eindimensionale Monte-Carlo-Integration

Aufgrund des Gaußfaktors im Integranden werden `n_max` normalverteilte Zufallszahlen gezogen. Die Integration erfolgt dann durch eine gewichtete Summation über den restlichen Integranden, der an den Zufallszahlen ausgewertet wird. Um die Ergebnisse reproduzierbar zu machen, wird beim Initialisieren des Zufallsgenerators ein `seed` angegeben. Dieser Wert kann bei Bedarf entfernt werden, um die Variation des Ergebnisses für verschiedene Werte der Zufallszahlen zu untersuchen.

In [ ]:
def integral_mc(n_max):
    rng = np.random.default_rng(123456)
    random_numbers = rng.normal(size=n_max)
    int_result = np.ndarray.sum(random_numbers**2) / n_max
    return int_result

### Implementierung der Bedienelemente und Ausgabe des Integrationsergebnisses

Mit Hilfe des Schiebereglers lässt sich der folgende Parameter einstellen:
- `log_n_max`: Zehnerlogarithmus der Anzahl der zur Integration verwendeten Zufallszahlen

Neben dem Resultat der Monte-Carlo-Integration wird auch der tatsächliche relative Fehler ausgegeben, der hier gleich dem absoluten Fehler ist.

In [ ]:
widget_dict = {"log_n_max":
               widgets.IntSlider(
                   value=2, min=2, max=8, step=1,
                   description=r"$\log_{10}(N)$")
               }

@interact(**widget_dict)
def print_mc_1d(log_n_max):
    n_max = 10**log_n_max
    int_result_mc = integral_mc(n_max)
    print(int_result_mc)
    print("Tatsächlicher relativer Fehler: "
          f"{abs(int_result_mc-1):8.2e}")

### Mehrdimensionale Monte-Carlo-Integration

Zur Erweiterung auf ein mehrdimensionales Integral wird hier ein zweidimensionales NumPy-Array normalverteilter Zufallszahlen erzeugt, dessen Achse 1 der Dimension des Integrals entspricht.

In [ ]:
def integral_mc_ndim(n_dim, n_max):
    rng = np.random.default_rng(123456789)
    random_numbers = rng.normal(size=(n_max, n_dim))
    individual_averages = np.sum(
        random_numbers**2, axis=0) / n_max
    int_result = np.sum(individual_averages)
    return int_result

### Implementierung der Bedienelemente und Ausgabe des Integrationsergebnisses

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_dim`: Dimension des Integrals
- `log_n_max`: Zehnerlogarithmus der Anzahl der zur Integration verwendeten Zufallszahlen je Dimension

Neben dem Resultat der Monte-Carlo-Integration wird auch der tatsächliche relative Fehler ausgegeben. Da die Berechnung für größere Werte von `n_dim` etwas länger dauern kann, wird ggf. ein entsprechender Hinweis ausgegeben.

In [ ]:
widget_dict = {"n_dim":
               widgets.IntSlider(
                   value=1, min=1, max=20, step=1,
                   description=r"$n_\text{dim}$",
                   continuous_update=False),
               "log_n_max":
               widgets.IntSlider(
                   value=2, min=2, max=7, step=1,
                   description=r"$\log_{10}(N)$")
               }

@interact(**widget_dict)
def print_mc_nd(n_dim, log_n_max):
    out = widgets.Output()
    display(out)
    out.append_stdout("Bitte etwas Geduld...")

    n_max = 10**log_n_max
    int_result = integral_mc_ndim(n_dim, n_max)

    out.clear_output()
    with out:
        print(int_result)
        print("Tatsächlicher relativer Fehler: "
              f"{abs(int_result-n_dim) / n_dim:8.2e}")